In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

In [3]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

# Uncomment below for GPU
#model = model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [5]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # Uncomment below for GPU
        #data, target = data.cuda(), target.cuda()
        
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [6]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        
        # Uncomment below for GPU
        #data, target = data.cuda(), target.cuda()
        
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
for epoch in range(1, 10):
    train(epoch)
    test()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303635
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.292724
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.288792
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.279552
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.241232
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.237643
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.198836
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.140894
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.083631
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.957344
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.842908
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.690088
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.334901
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.156029
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.924870
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.953737
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.691978
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.664937
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.571473
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.749548
Train Epoch: 1 [12800/60000 (

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]



Test set: Average loss: 0.1975, Accuracy: 9453/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.153969
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.223802
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.182071
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.195183
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.321025
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.136971
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.212567
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.186753
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.266166
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.104072
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.114068
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.108478
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.172288
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.194815
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.117566
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.234624
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.170922
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.184807
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.204400
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.013342
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.234380
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.046996
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.140820
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.091526
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.062704
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.096294
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.099115
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.124333
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.054061
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.065069
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.151420
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.226872
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.036585
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.039331
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.038127
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.090157
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.062870
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.116804
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.216266


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.083967
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.096219
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.142527
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.011549
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.035511
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.050984
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.100477
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.023688
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.072633
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.054807
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.056780
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.091622
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.020333
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.064365
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.057407
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.082123
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.140623
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.190150
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.045506
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.057212


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.043007
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.018735
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.078337
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.063703
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.064546
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.090790
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.072741
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.034131
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.012335
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.093150
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.084164
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.052168
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.026792
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.036661
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.030858
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.165901
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.086413
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.057873
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.054635
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.106189


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.087266
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.034197
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.013548
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.013444
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.026935

Test set: Average loss: 0.0585, Accuracy: 9826/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.020439
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.013726
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.070934
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.026771
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.093844
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.119249
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.032005
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.012938
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.166744
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.064015
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.026047
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.018088
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.041391
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.035768
Train Epoch: 9 [